In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import torch

In [3]:
from storage.har_datasets import HARTHDataset, UCI_HARDataset
from s3ts.api.dms.har_datasets import LDFDataset, DFDataset
from storage.label_mappings import *

In [4]:
ds = UCI_HARDataset("./datasets/UCI-HAR/", split="train", wsize=64, normalize=True)
meds = np.load("./datasets/UCI-HAR/medoids.npz")
dfds = DFDataset(ds, patterns=meds, w=0.1, dm_transform=None)

In [5]:
len(dfds)

469248

In [6]:
data_split = {
    "train": lambda x: x<350000,
    "val": lambda x: (x>=350000) * (x<400000),
    "test": lambda x: x>=400000
}

dm = LDFDataset(dfds, data_split=data_split, batch_size=16, random_seed=42, num_workers=8)

In [7]:
len(dm.ds_train) + len(dm.ds_val) + len(dm.ds_test)

469248